In [ ]:
#----------------CNN TUNNING TRIAL-------------------------

import pandas as pd
import numpy as np
import keras_tuner as kt
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, LSTM
from tensorflow.keras.layers import Dense, Dropout, Flatten, Multiply, Permute, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score


# ---------------------- LOAD & PREPROCESS DATA ----------------------
attack_free = pd.read_csv("Attack_free new.csv")[0:2369397]
dos = pd.read_csv("DoS_Attack_new.csv")[0:656578]
fuzzy = pd.read_csv("Fuzzy_Attack_New.csv")[0:591989]
impersonation = pd.read_csv("Impersonation_Attack_New.csv")[0:995471]

for df, label in zip([attack_free, dos, fuzzy, impersonation], [0, 1, 2, 3]):
    df['label'] = label

df = pd.concat([attack_free, dos, fuzzy, impersonation], ignore_index=True)
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])

for col in df.columns[:-1]:
    df[col] = df[col].apply(lambda x: int(str(x), 16) if isinstance(x, str) and all(c in '0123456789abcdefABCDEF' for c in str(x)) else x)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(inplace=True)

X = df.drop(columns='label').values
y = df['label'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], X.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

y_train_cat = to_categorical(y_train, num_classes=4)
y_test_cat = to_categorical(y_test, num_classes=4)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

# ---------------------- CUSTOM ATTENTION ----------------------
def attention_layer(inputs):
    attention = Dense(inputs.shape[1], activation='softmax')(inputs)
    attention = Permute((2, 1))(attention)
    attention = Lambda(lambda x: K.mean(x, axis=1))(attention)
    attention = Dense(inputs.shape[-1], activation='sigmoid')(attention)
    return Multiply()([inputs, attention])

# ---------------------- HYPERMODEL ----------------------
def build_cnn_bilstm_model(hp):
    input_layer = Input(shape=(X.shape[1], 1))

    x = Conv1D(
        filters=hp.Int('conv_filters', 32, 128, step=32),
        kernel_size=hp.Choice('kernel_size', [3, 5]),
        activation='relu'
    )(input_layer)
    x = MaxPooling1D(pool_size=2)(x)

    x = Bidirectional(LSTM(
        hp.Int('lstm_units', 32, 128, step=32),
        return_sequences=True
    ))(x)

    x = attention_layer(x)
    x = Flatten()(x)

    x = Dense(hp.Int('dense_1', 64, 256, step=64), activation='relu')(x)
    x = Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1))(x)

    x = Dense(hp.Int('dense_2', 32, 128, step=32), activation='relu')(x)
    x = Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1))(x)

    output = Dense(4, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ---------------------- HYPERBAND TUNING ----------------------
tuner = kt.Hyperband(
    build_cnn_bilstm_model,
    objective='val_accuracy',
    max_epochs=100,
    factor=3,
    directory='cnn_bilstm_tuning',
    project_name='CAN_IDS_CNN_BiLSTM'
)

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

tuner.search(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=100,
    batch_size=256,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# ---------------------- TRAIN BEST MODEL ----------------------
best_hps = tuner.get_best_hyperparameters(1)[0]
print("\n🔧 Best Hyperparameters Found:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=100,
    batch_size=256,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

loss, acc = best_model.evaluate(X_test, y_test_cat)
print(f"\n Tuned CNN+BiLSTM Accuracy: {acc:.4f}")
best_model.save("tuned_CNN_BiLSTM_model.h5")


#---------------------- AUROC CALCULATION ----------------------



# Binarize the test labels
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])

# Predict class probabilities
y_score = model.predict(X_test)

# Compute AUROC for each class
auroc_per_class = roc_auc_score(y_test_bin, y_score, average=None)

# Print AUROC for each class
class_names = ['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']
print("\n AUROC Score per Class:")
for i, score in enumerate(auroc_per_class):
    print(f"{class_names[i]}: AUROC = {score:.4f}")

